# COS 473 Final Project: ZKSwap

The main objective of this project is to implement basic functions of a DEX to manage a liquidity tool and enable exchanges between reserves of two synthetic tokens on Polygon zkEVM to achieve fast, low-gas, zero-knowledge proof transactions. Specifically, the project aims to:
1. Implement the basic functions of DEX based on Assignment 4 framework.
2. Deploy Swap.sol contract on Goerli as well as on [Polygon zkEVM](https://polygon.technology/polygon-zkevm) along with contracts to the synthetic tokens.
3. Initialize contract with liquidity of both tokens, so that users are able to <br>
    a. add liquidity <br>
    b. remove liquidity <br>
    c. swap some amount of token 0 with token 1 and vice versa <br>
4. Evaluate and analyze the performance of the swap on zkEVM in terms of accuracy, gas fee, and speed.

## Step 1: Create Swap.sol

## Swap interface
Basic functions of DEXs in `contracts/Swap.sol` to manage a liquidity pool made up of reserves of two sAsset tokens and enable exchanges between them. Please follow the below specifications and the interfaces defined in `contracts/interfaces/ISwap.sol`. The protocol in this swap is derived from [Uniswap v2](https://docs.uniswap.org/protocol/V2/introduction).

### State variables

* `token0` / `token1`: addresses of a pair of sAsset tokens
* `reserve0` / `reserve1`: quantity of each sAsset token in the pool
* `totalShares`: the total amount of shares owned by all liquidity providers
* `shares`: a mapping from the address of a liquidity provider to the number of shares owned by the liquidity provider. `shares[LP] / totalShares` represents the relative proportion of total reserves that each liquidity provider has contributed

### Functions
There are some functions already implemented for the initial setup.

* `init` is used by the first liquidity provider (in our project it should be the owner of the contract) to deposit both tokens with equal values. The ratio of tokens defines the initial exchange rate and reflects the price of two tokens in the global market as the liquidity provider believes. The amount of initial shares follows [Uniswap v2 (section 3.4)](https://uniswap.org/whitepaper.pdf) and is set to be equal to the geometric mean of the amounts deposited: `shares = sqrt(amount0 * amount1)`.
* `sqrt` is a helper function to calculate square root.
* `getReserves` is a view function that returns the reserves of two tokens.
* `getTokens` is a view function that returns the addresses of two tokens.
* `getShares` is a view function that returns the number of shares owned by the given address.
* `addLiquidity` is used by future liquidity providers to deposit tokens, and will generate new shares based on the token amount in the deposit w.r.t. the pool. Adding liquidity requires an equivalent value of two tokens. Callers need to specify the amount of token 0 they want to deposit (`amount0`) and the amount of token 1 required to be added (`amount1`) is determined using the reserve rate at the moment of their deposit, i.e.,`amount1 = reserve1 * amount0 / reserve0`. And the amount of shares received by the liquidity provider is: `new_shares = total_shares * amount0 / reserve0`.
* `token0To1` / `token1To0` are the functions for converting token 0/1 to token 1/0 while maintaining the relationship `reserve0 * reserve1 = invariant`. The input specifies the number of source tokens sent to the smart contract, the function then computes the number of target tokens sent to the caller based on the current price rate and the input (after subtracting the 0.3% protocol fee). 

## Step 2: Deploy
* compile code into [Remix](https://remix.ethereum.org) and deploy on Goerli Testnet, verify contract
* deploy on Polygon zkEVM (several times might be necessary), verify contract
* make sure all other necessary files are included by copying them from our [GitHub repo](https://github.com/AllisonShen/Zkswap)

In [ ]:
'''
// SPDX-License-Identifier: MIT
pragma solidity >=0.8.0 <0.9.0;
import "@openzeppelin/contracts/access/Ownable.sol";
import "./interfaces/ISwap.sol";
import "./sAsset.sol";
import "@openzeppelin/contracts/utils/math/SafeMath.sol";

contract Swap is Ownable, ISwap {

    address token0;
    address token1;
    uint reserve0;
    uint reserve1;
    mapping (address => uint) shares;
    uint public totalShares;

    constructor(address addr0, address addr1) {
        token0 = addr0;
        token1 = addr1;
    }

    function init(uint token0Amount, uint token1Amount) external override onlyOwner {
        require(reserve0 == 0 && reserve1 == 0, "init - already has liquidity");
        require(token0Amount > 0 && token1Amount > 0, "init - both tokens are needed");
        
        require(sAsset(token0).transferFrom(msg.sender, address(this), token0Amount));
        require(sAsset(token1).transferFrom(msg.sender, address(this), token1Amount));
        reserve0 = token0Amount;
        reserve1 = token1Amount;
        totalShares = sqrt(token0Amount * token1Amount);
        shares[msg.sender] = totalShares;
    }

    // https://github.com/Uniswap/v2-core/blob/v1.0.1/contracts/libraries/Math.sol
    function sqrt(uint y) internal pure returns (uint z) {
        if (y > 3) {
            z = y;
            uint x = y / 2 + 1;
            while (x < z) {
                z = x;
                x = (y / x + x) / 2;
            }
        } else if (y != 0) {
            z = 1;
        }
    }

    function getReserves() external view returns (uint, uint) {
        return (reserve0, reserve1);
    }

    function getTokens() external view returns (address, address) {
        return (token0, token1);
    }

    function getShares(address LP) external view returns (uint) {
        return shares[LP];
    }

    /* TODO: implement your functions here */

    function addLiquidity(uint token0Amount) external override {
        require(reserve0 != 0, "addLiquidity - reserve0 can not be 0.");
        // find token1Amount
        uint token1Amount = (reserve1 * token0Amount) / reserve0;
        // find current shares and add new shares
        uint new_shares = (totalShares * token0Amount) / reserve0;

        // perform transfer
        require(sAsset(token0).transferFrom(msg.sender, address(this), token0Amount));
        require(sAsset(token1).transferFrom(msg.sender, address(this), token1Amount));

        reserve0 += token0Amount;
        reserve1 += token1Amount;
        shares[msg.sender] += new_shares;
        totalShares += new_shares;
    }

    function removeLiquidity(uint withdrawShares) external override {
        require(withdrawShares <= shares[msg.sender], "Sender doesn't have enough shares.");
        require(totalShares > 0, "Insufficient totalShares amount.");
        // uint total_shares = shares[msg.sender] / totalShares;
        uint token0Amount = reserve0 * withdrawShares / totalShares;
        uint token1Amount = reserve1 * withdrawShares / totalShares;

        
        // perform transfer
        require(sAsset(token0).transfer(msg.sender, token0Amount));
        require(sAsset(token1).transfer(msg.sender, token1Amount));

        shares[msg.sender] -= withdrawShares;
        reserve0 -= token0Amount;
        reserve1 -= token1Amount;
        totalShares -= withdrawShares;
    }

    function token0To1(uint token0Amount) external override {
        // use buffer to avoid solidity cutting off decimals
        uint buffer = 100;
        uint token0_in = (token0Amount * 997) / 1000;
        uint num = (reserve0 * reserve1 * buffer);
        uint denom = (reserve0 + token0_in);
        uint quotient = num / denom;    
        uint token1_out = ((reserve1 * buffer) - quotient) / buffer;
        
        // perform transfer
        require(sAsset(token0).transferFrom(msg.sender, address(this), token0Amount));
        require(sAsset(token1).transfer(msg.sender, token1_out));

        // update reserves
        reserve0 += token0Amount;
        reserve1 -= token1_out;
    }

    function token1To0(uint token1Amount) external override {
        // use buffer to avoid solidity cutting off decimals
        uint buffer = 100;
        uint token1_in = (token1Amount * 997) / 1000;
        uint num = (reserve0 * reserve1 * buffer);
        uint denom = (reserve1 + token1_in);
        uint quotient = num / denom;    
        uint token0_out = ((reserve0 * buffer) - quotient) / buffer;
        
        // perform transfer
        require(sAsset(token1).transferFrom(msg.sender, address(this), token1Amount));
        require(sAsset(token0).transfer(msg.sender, token0_out));

        // update reserves
        reserve0 -= token0_out;
        reserve1 += token1Amount;
    }
}
'''

## Addresses of Synthetic Tokens

Here are the addresses of the Synthetic Tokens and the Swap contract in both Testnets for testing purposes.

| | Goerli Testnet | Polygon zkEVM Testnet |
| --- | --- | --- |
| sBNB | 0xa25ce7101271651a0333788a294d825b8e09e275 | 0x6d1d267fb66c82a904a6b31bc82f39f08f0fb175 |
| sTSLA | 0x51a6115f779a88a4fff8766039d99af8d95e07d3 | 0x44ebc699f42a2d53fe68d20d00130cf1ec0fad24 |
| Swap | 0x1d0a5bdb90b9d4f1bb5b17516a07f031cc903099 | 0xce3fcbafeafeb895aa6a6e4ca67a8a7d7957e42e |

## Polygon zkEVM Bridge
* use the [Polyzon zkEVM Bridge](https://wallet.polygon.technology/?redirectOnConnect=%2FzkEVM-Bridge%2Fbridge) to transfer assets from Goerli to Polygon so that you can test the functions.

## Step 3: Testing!
* Next, we performed timing analysis on each of our methods in the test suite to determine the timing differences between the two networks.
* Test suite is below

In [ ]:
'''

const { performance } = require('perf_hooks');
const config = require('config');
const fs = require('fs');

var receipts = new Map();
var elapsed_time_map = new Map();
var gas_used_map = new Map();
var instances = {};
var owner_accounts = {};
var function_name;
var elapsed_time;
var current_receipt;
var shares;

const amount = 100000 * 10 ** 8;
const tokenSent = 1000 * 10 ** 8;

const contracts_to_deploy = ['sBNB', 'sTSLA', 'Swap']
const network_name = process.env.network;

var map_to_file = function (file_name, map) {
    let json_obj = {};
    json_obj.map = {};
    for (let [key, value] of map)
        json_obj.map[key] = value;
    let json_str = JSON.stringify(json_obj);
    fs.writeFileSync(file_name, json_str);
}

var save_data = function (data_map, key, value) {
    if (key) {
        var data_list = [];
        if (data_map.has(key)) {
            data_list = data_map.get(key);
        }
        data_list.push(value);
        data_map.set(key, data_list);
    }
}

var setup = async function (accounts) {
    console.log('network_name = ', network_name);
    if (network_name == 'development') {
        var contracts = {}
        owner_accounts = accounts;
        for (contract_name of contracts_to_deploy) {
            contracts[contract_name] = artifacts.require(contract_name)
            instances[contract_name] = await contracts[contract_name].deployed();
        }
    } else {
        network = config.get(network_name);
        console.log('network = ', network);
        owner = config.get('owner');
        owner_accounts = owner.accounts.split(', ');
        for (contract_name of contracts_to_deploy) {
            let contract = artifacts.require(contract_name);
            let contract_address = network[contract_name];
            contract.setProvider(web3.currentProvider);
            instances[contract_name] = await contract.at(contract_address);
        }
    }
    console.log('owner_accounts = ', owner_accounts);
}

afterEach(() => {
    if (function_name != 'setup') {
        save_data(elapsed_time_map, function_name, elapsed_time);
        if (current_receipt) {
            save_data(gas_used_map, function_name, current_receipt.gasUsed);
            receipts.set(current_receipt.transactionHash, current_receipt);
            current_receipt = null;
        }
    }
});

after(() => {
    var timestamp = Date.now();
    const dir = 'data_' + network_name;
    if (!fs.existsSync(dir)) {
        fs.mkdirSync(dir);
    }
    map_to_file(dir + '/receipts-' + timestamp + '.json', receipts);
    map_to_file(dir + '/elapsed-times-' + timestamp + '.json', elapsed_time_map);
    map_to_file(dir + '/gas-used-' + timestamp + '.json', gas_used_map);
});

contract("Performance Tests", async accounts => {

    it("Test 00: setup", async () => {
        function_name = 'setup';
        var start = performance.now();
        await setup(accounts);
        elapsed_time = performance.now() - start;
    });

    it("Test 01: getTokens", async () => {
        function_name = 'getTokens';
        var start = performance.now();
        await instances['Swap'].getTokens.call()
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 02: approve sBNB", async () => {
        function_name = 'approve';
        var start = performance.now();
        await instances['sBNB'].approve(instances['Swap'].address, amount * 2)
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 03: approve sTSLA", async () => {
        function_name = 'approve';
        var start = performance.now();
        await instances['sTSLA'].approve(instances['Swap'].address, amount * 2)
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 04: init", async () => {
        function_name = 'init';
        var start = performance.now();
        await instances['Swap'].init(amount, amount)
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 05: getReserves", async () => {
        function_name = 'getReserves';
        var start = performance.now();
        await instances['Swap'].getReserves.call()
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 06: addLiquidity", async () => {
        function_name = 'addLiquidity';
        var start = performance.now();
        await instances['Swap'].addLiquidity(amount)
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 07: transfer sBNB", async () => {
        function_name = 'transfer';
        var start = performance.now();
        await instances['sBNB'].transfer(owner_accounts[1], tokenSent)
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 08: approve sBNB", async () => {
        function_name = 'approve';
        var start = performance.now();
        await instances['sBNB'].approve(instances['Swap'].address, tokenSent, { from: owner_accounts[1] })
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 09: token0To1", async () => {
        function_name = 'token0To1';
        var start = performance.now();
        await instances['Swap'].token0To1(tokenSent, { from: owner_accounts[1] })
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 10: balanceOf", async () => {
        function_name = 'balanceOf';
        var start = performance.now();
        await instances['sTSLA'].balanceOf.call(owner_accounts[1])
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 11: transfer sTSLA", async () => {
        function_name = 'transfer';
        var start = performance.now();
        await instances['sTSLA'].transfer(owner_accounts[2], tokenSent)
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 12: approve sTSLA", async () => {
        function_name = 'approve';
        var start = performance.now();
        await instances['sTSLA'].approve(instances['Swap'].address, tokenSent, { from: owner_accounts[2] })
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 13: token1To0", async () => {
        function_name = 'token1To0';
        var start = performance.now();
        await instances['Swap'].token1To0(tokenSent, { from: owner_accounts[2] })
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 14: getShares", async () => {
        function_name = 'getShares';
        var start = performance.now();
        shares = await instances['Swap'].getShares.call(owner_accounts[0])
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });

    it("Test 15: removeLiquidity", async () => {
        function_name = 'removeLiquidity';
        var start = performance.now();
        await instances['Swap'].removeLiquidity(shares)
            .on("receipt", function (receipt) {
                current_receipt = receipt;
            })
            .on("error", function (error) {
                console.error('error: ', error);
            });
        elapsed_time = performance.now() - start;
    });
});

'''